# FiberSpace

In [1]:
import requests
import pandas as pd
import numpy as np
from skimage import io
from sklearn.decomposition import PCA
from scipy.spatial import ConvexHull
import bokeh
from bokeh.plotting import figure, show, output_notebook, output_file

from IPython.display import Image, HTML, Javascript
from coffeetools import coffee
output_notebook()

Loading BokehJS ...

In [2]:
# Must make all photos public on Flickr for this cell to work.

import flickrapi
api_key = u'e8366dedadcfc81dcd0c761492b91827'
api_secret = u'f8bc4e6c4e7bf4ff'

flickr = flickrapi.FlickrAPI(api_key, api_secret, format='json')
photos = flickr.photos.search(user_id='127699624@N02', per_page='100')
df_flickr = pd.read_json(photos)
df_flickr = pd.DataFrame(df_flickr['photos']['photo'])
df_flickr.to_csv('/Users/Imperssonator/CC/Zoom_Imgs/flickr.csv')
df_flickr.head()

,farm,id,isfamily,isfriend,ispublic,owner,secret,server,title
0,6,30642437092,0,0,1,127699624@N02,3d090af106,5738,MFU 0.25 spin coating 001_2um.002
1,6,30758994195,0,0,1,127699624@N02,821fd60309,5513,MFU 0.1 spin coating 001_2um.001
2,6,30642437132,0,0,1,127699624@N02,3c90407f91,5468,MFU 0.1 spin coating 002_2um.001
3,6,30758994255,0,0,1,127699624@N02,1e192c2be1,5465,MFU 0.6 spin coating 001_2um.001
4,6,30670369541,0,0,1,127699624@N02,37fed69b13,5482,SonAge Sol3 blade 2 V3.002 rot


In [3]:
df_flickr = pd.read_csv('/Users/Imperssonator/CC/Zoom_Imgs/flickr.csv')
df_flickr.head()

,Unnamed: 0,farm,id,isfamily,isfriend,ispublic,owner,secret,server,title
0,0,6,30642437092,0,0,1,127699624@N02,3d090af106,5738,MFU 0.25 spin coating 001_2um.002
1,1,6,30758994195,0,0,1,127699624@N02,821fd60309,5513,MFU 0.1 spin coating 001_2um.001
2,2,6,30642437132,0,0,1,127699624@N02,3c90407f91,5468,MFU 0.1 spin coating 002_2um.001
3,3,6,30758994255,0,0,1,127699624@N02,1e192c2be1,5465,MFU 0.6 spin coating 001_2um.001
4,4,6,30670369541,0,0,1,127699624@N02,37fed69b13,5482,SonAge Sol3 blade 2 V3.002 rot


In [4]:
# Flickr image url format:
# https://farm{farm-id}.staticflickr.com/{server-id}/{id}_{secret}.jpg

def make_url(img_path,df_flickr):
    row = df_flickr[df_flickr['title']==img_path].iloc[0]
    url = 'https://farm'+str(row['farm'])+'.staticflickr.com/'+ \
    str(row['server'])+'/'+str(row['id'])+ \
    '_'+str(row['secret'])+'.jpg'
    return url


In [5]:
import pandas as pd

the_csv_file = 'file:///Users/Imperssonator/Google Drive/My AFM/All Good Images/summary_stats_all.csv'
df = pd.read_csv(the_csv_file)
df.head()

,ImageName,Sfull,CorrLen,MeanLength,MeanWidth,LengthDensity,ImageSize,Process
0,MFU 0.1 spin coating 001_2um.001,0.873260,622.3635,361.6864,14.4767,0.015281,2,MFU
1,MFU 0.1 spin coating 002_2um.001,0.811830,402.2616,344.7610,15.1403,0.014652,2,MFU
2,MFU 0.25 spin coating 001_2um.002,0.337630,506.4571,164.7038,15.5168,0.011035,2,MFU
3,MFU 0.6 spin coating 001_2um.001,0.252630,495.0272,190.1355,14.5377,0.011836,2,MFU
4,10%_2min,0.087859,322.8267,148.7525,21.1618,0.009223,2,Son+2MP


In [6]:
df_num = df[['Sfull','CorrLen','MeanLength','MeanWidth','LengthDensity']]
df_num.corr()

,Sfull,CorrLen,MeanLength,MeanWidth,LengthDensity
Sfull,1.000000,-0.301974,0.396393,-0.337376,0.427051
CorrLen,-0.301974,1.000000,0.404261,0.468947,-0.448276
MeanLength,0.396393,0.404261,1.000000,0.379466,-0.238552
MeanWidth,-0.337376,0.468947,0.379466,1.000000,-0.895977
LengthDensity,0.427051,-0.448276,-0.238552,-0.895977,1.000000


In [7]:
df['DotSize'] = df['ImageSize']*2
df['file_url'] = df['ImageName'].apply(lambda x: make_url(x,df_flickr))
df['file_url'][10]

'https://farm6.staticflickr.com/5459/30721965636_f8d7e203bd.jpg'

In [8]:
colors = {}
class_to_color = 'Process'
for c in df[class_to_color].unique():
    colors[c] = np.random.randint( 0,256, (1,3))
    df.loc[df[class_to_color]==c,'color'] = '#' + ''.join([ '%02x'%s for s in colors[c][0]] )
    
df['color'].head()

0    #a8868d
1    #a8868d
2    #a8868d
3    #a8868d
4    #b93e20
Name: color, dtype: object

In [11]:
def zoom_img_plot(df,x_col,y_col,x_label='X',y_label='Y',plot_width = 800,plot_height=600):
    p=figure(responsive=False, x_axis_label = x_label, y_axis_label = y_label,width = plot_width, height=plot_height)
    
    x_range = df[x_col].max()-df[x_col].min()
    y_range = df[y_col].max()-df[y_col].min()

    cds = bokeh.models.ColumnDataSource(
        df
    )
    # cds.data['volume_fraction'] = [ v*5 for v in cds.data['volume_fraction'] ]

    circles = p.circle(
        source=cds,
        x=x_col,
        y=y_col,
        fill_color='color',
        size='DotSize',
        fill_alpha=.6
    )

    p.add_tools(
        bokeh.models.HoverTool(
            tooltips=[
                ('S full', '@Sfull'),
                ('Decay Length (nm)','@CorrLen'),
                ('Image Size (um)','@ImageSize'),
                ('Process','@Process')
            ]
        )
    )

    images = p.image_url(
        source=cds,
        x=x_col,
        y=y_col,
        w=x_range*0.02,
        h=y_range*0.02*plot_width/plot_height,
        url='file_url',
        anchor='center',
        global_alpha=.001
    )

    # mu = [.5]
    # std = np.sqrt(-1*np.diff(mu)**2/(2*np.log(.2)))

    p.x_range.callback = bokeh.models.CustomJS( args= {
            'x_axis': p.x_range,
            'y_axis': p.y_range,
            'images': images.glyph,
            'cds': cds,
            'points': p.renderers[5].glyph
        }, 
        code=coffee.compile("""
        ddata = Bokeh._.max(cds.get('data')[points.get('x').field]) - Bokeh._.min(cds.get('data')[points.get('x').field])
        dx = x_axis.get('end') - x_axis.get('start')
        if dx/ddata > .0001
            images.set 'global_alpha', .001 + (.85)*Math.exp( -1*Math.pow(dx/ddata-.0001,2)/.014)
        """,bare=True)
    )
    
    # patches = []
    # for c in df['class'].unique():
    #     i = df.loc[df['class']==c].index.values
    #     h = ConvexHull(df.iloc[i][['pca_x','pca_y']])
    #     pp = p.patch(**{
    #             'fill_color': df.iloc[i[0]].color,
    #             'fill_alpha': .4,
    #             'x': df.iloc[i].pca_x.values[h.vertices],
    #             'y': df.iloc[i].pca_y.values[h.vertices],
    #             'legend': c,
    #         })

    # gds=bokeh.models.ColumnDataSource(g)
    # class_scale = p.oval( 
    #     source=gds, 
    #     x='pca_x.mean', 
    #     width='pca_x.std',
    #     y='pca_y.mean',
    #     height='pca_y.std',
    #     fill_color='color.min',
    #     fill_alpha = .6,
    # )

    # p.add_tools(
    #     bokeh.models.TapTool( 
    #         renderers=[pp],
    #         callback=bokeh.models.CustomJS(
    #                 code=coffee.compile("""
    #                 console.log 'click'
    #                 """,bare=True)
    #             )
    #     )
    # )
    return p
    

In [12]:
p1 = zoom_img_plot(df,'Sfull','LengthDensity',x_label='Sfull',y_label='Length Density (1/nm)')
show(p1)

In [87]:
p2 = zoom_img_plot(df,'MeanLength','CorrLen',x_label='Average Fiber Length (nm)',y_label='Decay Length (nm)')
show(p2)

In [88]:
p2 = zoom_img_plot(df,'MeanWidth','LengthDensity',x_label='Average Fiber Width (nm)',y_label='Fiber Length Density (1/nm)')
show(p2)

In [83]:
output_file('FiberSpace.html')